# Step 3: Storage (The Library)

In [13]:
import pandas as pd
import sqlite3
import os

INPUT_PATH = '../data/books_cleaned.csv'
DB_PATH = '../data/books.db'

df = pd.read_csv(INPUT_PATH, encoding="latin1")
print(f"Loaded {len(df)} rows.")

Loaded 10921 rows.


**Creating SQLite Database and Schema**

In [14]:
# Connect to SQLite
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Create table
# Mapping dataframe columns to SQL columns.
create_table_query = """
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    author TEXT,
    year SMALLINT,
    edition TEXT,
    publisher TEXT,
    isbn TEXT UNIQUE,
    description TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()
print("Table created.")

Table created.


**Inserting Data**

In [15]:
if not df.empty:
    # Map columns
    
    data_to_insert = pd.DataFrame()
    data_to_insert['title'] = df['Title'] if 'Title' in df.columns else None
    data_to_insert['author'] = df['Author/Editor'] if 'Author/Editor' in df.columns else None
    data_to_insert['year'] = df['Year'] if 'Year' in df.columns else None
    data_to_insert['edition'] = df['Ed./Vol.'] if 'Ed./Vol.' in df.columns else None
    data_to_insert['publisher'] = df['Place & Publisher'] if 'Place & Publisher' in df.columns else None
    data_to_insert['isbn'] = df['clean_isbn']
    data_to_insert['description'] = df['clean_description']
    
    # Use pandas to_sql
    try:
        data_to_insert.to_sql('books', conn, if_exists='replace', index=False)
        print(f"Inserted {len(data_to_insert)} rows into 'books' table.")
    except Exception as e:
        print(f"Error inserting data: {e}")

conn.close()

Inserted 10921 rows into 'books' table.


**Verifying**

In [16]:
conn = sqlite3.connect(DB_PATH)
test_df = pd.read_sql_query("SELECT * FROM books LIMIT 5", conn)
print(test_df)
conn.close()

                                               title  \
0  Network design : management and technical pers...   
1  Workflow Management Systems for Process Organi...   
2  Conceptual modeling : current issues and futur...   
3  Computer-Supported Cooperative Work : introduc...   
4                             Multimedia Cartography   

                     author    year edition                   publisher  \
0  Mann-Rubinson, Teresa C.  1999.0    None      Boca Raton: CRC Press,   
1            Schael, Thomas  1998.0    None  New York: Springer-Verlag,   
2            Chen, Peter P.  1999.0    None           Berlin: Springer,   
3          Borghoff, Uwe M.  2000.0    None  New York: Springer-Verlag,   
4      Peterson, Michael P.  1999.0    None  New York: Springer-Verlag,   

          isbn                                        description  
0    849334047  Network Design outlines the fundamental princi...  
1   354065304X  "This book introduces the perspective of custo...  
2  9.783